# Finite Automata


Finite Automata are mathematical models used in computational theory to represent systems with a finite number of states and transitions between these states. Here, we'll discuss various types and operations related to Finite Automata.

#Deterministic Finite Automata (DFA)

Deterministic Finite Automata (DFA) are a type of finite automaton where for each state and input symbol, there is exactly one next state.

In [1]:
class DFA:
    def __init__(self, states, alphabet, transition_function, initial_state, accepting_states):
        self.states = states
        self.alphabet = alphabet
        self.transition_function = transition_function
        self.current_state = initial_state
        self.accepting_states = accepting_states

    def transition(self, input_symbol):
        if input_symbol in self.alphabet:
            self.current_state = self.transition_function[self.current_state][input_symbol]

    def is_accepting(self):
        return self.current_state in self.accepting_states

# Testing DFA
def test_dfa():
    dfa = DFA(states={'q0', 'q1'},
              alphabet={'0', '1'},
              transition_function={'q0': {'0': 'q1', '1': 'q0'},
                                   'q1': {'0': 'q0', '1': 'q1'}},
              initial_state='q0',
              accepting_states={'q1'})
    inputs = "00110"
    for symbol in inputs:
        dfa.transition(symbol)
    assert dfa.is_accepting()
    print("DFA test passed.")

test_dfa()


DFA test passed.


#Non-Deterministic Finite Automata (NFA)

Non-Deterministic Finite Automata (NFA) are finite automata where for a given state and input symbol, there can be multiple next states.


In [2]:
class NFA:
    def __init__(self, states, alphabet, transition_function, initial_state, accepting_states):
        self.states = states
        self.alphabet = alphabet
        self.transition_function = transition_function
        self.current_states = {initial_state}
        self.accepting_states = accepting_states

    def transition(self, input_symbol):
        next_states = set()
        for state in self.current_states:
            if input_symbol in self.transition_function[state]:
                next_states.update(self.transition_function[state][input_symbol])
        self.current_states = next_states

    def is_accepting(self):
        return bool(self.current_states.intersection(self.accepting_states))

# Testing NFA
def test_nfa():
    nfa = NFA(states={'q0', 'q1'},
              alphabet={'0', '1'},
              transition_function={'q0': {'0': {'q0', 'q1'}, '1': {'q0'}},
                                   'q1': {'1': {'q1'}}},
              initial_state='q0',
              accepting_states={'q1'})
    inputs = "00110"
    for symbol in inputs:
        nfa.transition(symbol)
    assert nfa.is_accepting()
    print("NFA test passed.")

test_nfa()


NFA test passed.


#NFA to DFA Transformation

Transformation of NFA to DFA involves converting a non-deterministic finite automaton into an equivalent deterministic finite automaton. This conversion allows for easier analysis and implementation. The subset construction algorithm is commonly used for this purpose.



In [3]:
class NFA:
    def __init__(self, states, alphabet, transition_function, initial_state, accepting_states):
        self.states = states
        self.alphabet = alphabet
        self.transition_function = transition_function
        self.initial_state = initial_state
        self.accepting_states = accepting_states

    def epsilon_closure(self, states):
        closure = set(states)
        stack = list(states)
        while stack:
            state = stack.pop()
            if state in self.transition_function and '' in self.transition_function[state]:
                epsilon_transitions = self.transition_function[state]['']
                for next_state in epsilon_transitions:
                    if next_state not in closure:
                        closure.add(next_state)
                        stack.append(next_state)
        return closure

    def move(self, states, symbol):
        move_states = set()
        for state in states:
            if state in self.transition_function and symbol in self.transition_function[state]:
                move_states.update(self.transition_function[state][symbol])
        return move_states

    def epsilon_closure_of_move(self, states, symbol):
        return self.epsilon_closure(self.move(states, symbol))

    def to_dfa(self):
        dfa_states = []
        dfa_transition_function = {}
        dfa_initial_state = frozenset(self.epsilon_closure({self.initial_state}))
        dfa_accepting_states = []

        unmarked_states = [dfa_initial_state]
        marked_states = set()

        while unmarked_states:
            current_dfa_state = unmarked_states.pop()
            marked_states.add(current_dfa_state)

            dfa_states.append(current_dfa_state)

            if any(state in self.accepting_states for state in current_dfa_state):
                dfa_accepting_states.append(current_dfa_state)

            for symbol in self.alphabet:
                move_states = self.epsilon_closure_of_move(current_dfa_state, symbol)
                if move_states:
                    dfa_next_state = frozenset(move_states)
                    if dfa_next_state not in marked_states:
                        unmarked_states.append(dfa_next_state)
                    dfa_transition_function.setdefault(current_dfa_state, {})[symbol] = dfa_next_state

        return DFA(dfa_states, self.alphabet, dfa_transition_function, dfa_initial_state, dfa_accepting_states)


class DFA:
    def __init__(self, states, alphabet, transition_function, initial_state, accepting_states):
        self.states = states
        self.alphabet = alphabet
        self.transition_function = transition_function
        self.initial_state = initial_state
        self.accepting_states = accepting_states

    def transition(self, input_symbol):
        self.current_state = self.transition_function[self.current_state][input_symbol]

    def is_accepting(self):
        return self.current_state in self.accepting_states


# Example usage:
nfa = NFA(states={'q0', 'q1'},
          alphabet={'0', '1'},
          transition_function={'q0': {'0': {'q0'}, '1': {'q1'}, '': {'q1'}},
                               'q1': {'1': {'q1'}}},
          initial_state='q0',
          accepting_states={'q1'})

dfa = nfa.to_dfa()

print("DFA States:", dfa.states)
print("DFA Transition Function:", dfa.transition_function)
print("DFA Initial State:", dfa.initial_state)
print("DFA Accepting States:", dfa.accepting_states)


DFA States: [frozenset({'q0', 'q1'}), frozenset({'q1'})]
DFA Transition Function: {frozenset({'q0', 'q1'}): {'0': frozenset({'q0', 'q1'}), '1': frozenset({'q1'})}, frozenset({'q1'}): {'1': frozenset({'q1'})}}
DFA Initial State: frozenset({'q0', 'q1'})
DFA Accepting States: [frozenset({'q0', 'q1'}), frozenset({'q1'})]


#Non-Deterministic Finite Automata with ε-transitions (NFA-ε)

NFA-ε is an extension of NFA where transitions can be made without consuming any input symbol. This allows for more expressive power in representing languages.


In [10]:
class NFA_E:
    def __init__(self, states, alphabet, transition_function, initial_state, accepting_states):
        self.states = states
        self.alphabet = alphabet
        self.transition_function = transition_function
        self.current_states = {initial_state}
        self.accepting_states = accepting_states

    def epsilon_closure(self, states):
        closure = set(states)
        stack = list(states)
        while stack:
            state = stack.pop()
            if state in self.transition_function and '' in self.transition_function[state]:
                epsilon_transitions = self.transition_function[state]['']
                for next_state in epsilon_transitions:
                    if next_state not in closure:
                        closure.add(next_state)
                        stack.append(next_state)
        return closure

    def transition(self, input_symbol):
        next_states = set()
        for state in self.current_states:
            if input_symbol in self.transition_function[state]:
                next_states.update(self.transition_function[state][input_symbol])
        self.current_states = self.epsilon_closure(next_states)

    def is_accepting(self):
        return bool(self.current_states.intersection(self.accepting_states))
# Testing NFA-ε
def test_nfa_e():
    nfa_e = NFA_E(states={'q0', 'q1'},
                  alphabet={'0', '1'},
                  transition_function={'q0': {'0': {'q0'}, '1': {'q1'}, '': {'q1'}},
                                       'q1': {'1': {'q1'}}},
                  initial_state='q0',
                  accepting_states={'q1'})
    input_string = "1"
    for symbol in input_string:
        nfa_e.transition(symbol)
    assert nfa_e.is_accepting(), f"The NFA-ε did not accept the input string '{input_string}'"
    print("NFA-ε test passed.")

test_nfa_e()


NFA-ε test passed.


#NFA-ε to DFA Transformation

Transformation of NFA-ε to DFA involves converting a non-deterministic finite automaton with ε-transitions into an equivalent deterministic finite automaton. This conversion is more complex than the standard NFA to DFA transformation due to the presence of ε-transitions.



In [13]:
class NFA_E:
    def __init__(self, states, alphabet, transition_function, initial_state, accepting_states):
        self.states = states
        self.alphabet = alphabet
        self.transition_function = transition_function
        self.initial_state = initial_state
        self.accepting_states = accepting_states

    def epsilon_closure(self, states):
        closure = set(states)
        stack = list(states)
        while stack:
            state = stack.pop()
            if state in self.transition_function and '' in self.transition_function[state]:
                epsilon_transitions = self.transition_function[state]['']
                for next_state in epsilon_transitions:
                    if next_state not in closure:
                        closure.add(next_state)
                        stack.append(next_state)
        return closure

    def move(self, states, symbol):
        move_states = set()
        for state in states:
            if state in self.transition_function and symbol in self.transition_function[state]:
                move_states.update(self.transition_function[state][symbol])
        return move_states

    def epsilon_closure_of_move(self, states, symbol):
        return self.epsilon_closure(self.move(states, symbol))

    def to_dfa(self):
        dfa_states = set()
        dfa_transition_function = {}
        unmarked_states = [frozenset(self.epsilon_closure({self.initial_state}))]

        while unmarked_states:
            current_dfa_state = unmarked_states.pop()
            dfa_states.add(current_dfa_state)

            for symbol in self.alphabet:
                next_states = set()
                for state in current_dfa_state:
                    next_states |= self.epsilon_closure_of_move({state}, symbol)
                if next_states:
                    next_states_closure = frozenset(self.epsilon_closure(next_states))
                    if next_states_closure not in dfa_states:
                        unmarked_states.append(next_states_closure)
                        dfa_states.add(next_states_closure)
                    dfa_transition_function[current_dfa_state, symbol] = next_states_closure

        dfa_accepting_states = {state for state in dfa_states if state.intersection(self.accepting_states)}

        return DFA(dfa_states, self.alphabet, dfa_transition_function, frozenset(self.epsilon_closure({self.initial_state})), dfa_accepting_states)




class DFA:
    def __init__(self, states, alphabet, transition_function, initial_state, accepting_states):
        self.states = states
        self.alphabet = alphabet
        self.transition_function = transition_function
        self.initial_state = initial_state
        self.accepting_states = accepting_states

    def transition(self, input_symbol):
        self.current_state = self.transition_function[self.current_state, input_symbol]

    def is_accepting(self):
        return self.current_state in self.accepting_states


# Example usage:
nfa_e = NFA_E(states={'q0', 'q1'},
              alphabet={'0', '1'},
              transition_function={'q0': {'0': {'q0'}, '1': {'q1'}, '': {'q1'}},
                                   'q1': {'1': {'q1'}}},
              initial_state='q0',
              accepting_states={'q1'})

dfa = nfa_e.to_dfa()

print("DFA States:", dfa.states)
print("DFA Transition Function:", dfa.transition_function)
print("DFA Initial State:", dfa.initial_state)
print("DFA Accepting States:", dfa.accepting_states)


DFA States: {frozenset({'q1'}), frozenset({'q0', 'q1'})}
DFA Transition Function: {(frozenset({'q0', 'q1'}), '0'): frozenset({'q0', 'q1'}), (frozenset({'q0', 'q1'}), '1'): frozenset({'q1'}), (frozenset({'q1'}), '1'): frozenset({'q1'})}
DFA Initial State: frozenset({'q0', 'q1'})
DFA Accepting States: {frozenset({'q1'}), frozenset({'q0', 'q1'})}
